# Note

## 년도별로 따로 불러오기 -> 나중에 concat 후 drop_duplicates 이용하여 가장 최신것만 남기기

In [70]:
import os
import pandas as pd
import pykrx
import pyautogui
import time
import numpy as np
from tqdm import tqdm
import pymysql
from selenium import webdriver
import shutil

class Data:
    def __init__(self):
        self.is_chrome = False
        self.path = None
    
    def setMaria(self, host='localhost', user='root', password='sa1234', db='jaemoo', charset='utf8'):
        self.connect = pymysql.connect(host=host, user=user, password=password,db=db, charset=charset)
        self.cur = self.connect.cursor()
        return
    
    def mariaInsertData(self, tablename, data):
        try:
            sql = "insert into {} values {};".format(tablename, data)
            self.cur.execute(sql)
        except Exception as e:
            print(sql)
            print(e.args)
        return

    def mariaCommitDB(self):
        self.connect.commit()
        return

    def mariaShowData(self, tablename):
        try:
            self.cur.execute('select * from '+ tablename)
            df = self.cur.fetchall()
            field_names = [i[0] for i in self.cur.description]
            df = pd.DataFrame(df, columns=field_names)
            return df
        except Exception as e:
            return None
        return
    
    def mariaSql(self, sql):
        self.cur.execute(sql)
        self.cur.fetchall()
        return

    def mariaCreateTable(self, tablename, columns, columns_type):
        try:
            sql = "CREATE TABLE {} ({} {} PRIMARY KEY".format(tablename, columns[0], columns_type[0])
            for i in range(1, len(columns)):
                sql += ", {} {}".format(columns[i], columns_type[i])
            sql += ');'
            self.cur.execute(sql)
        except Exception as e:
            print(sql)
            print(e)
            pass
        return
    
    def setChrome(self, visualize=True):
        chromedriver = 'chromedriver.exe'
        if not visualize:
            webdriver_options = webdriver.ChromeOptions()
            webdriver_options.add_argument('headless')
            self.driver = webdriver.Chrome(chromedriver, options=webdriver_options)
        else:
            self.driver = webdriver.Chrome(chromedriver)
        self.is_chrome = True
        return
    
    def enterFnDownloadURL(self):
        self.driver.get('https://www.fnguide.com/Fgdd/FinIndivCompTrend#gicode=A005930&conyn=1&termgb=D&acntcode=10')
        return

    def getFnCsvFile(self, filename):
        df = pd.read_csv(self.path + '/' + filename, encoding='utf-8')
        return df
    
    def modifyFnCsvFile(self, df):
        df.columns = df.iloc[7]
        df = df.drop(7).reset_index(drop=True)
        df = df.iloc[7:].reset_index(drop=True)
        cols_list = []
        for c in df[df.columns[0]]:
            if '  ' not in c:
                col1 = c.replace(' ', '')
                cols_list.append(col1)
            elif '    ' not in c:
                col2 = col1 + '_' + c.replace(' ', '')
                cols_list.append(col2)
            elif '      ' not in c:
                col3 = col2 + '_' + c.replace(' ', '')
                cols_list.append(col3)
            elif '        ' not in c:
                col4 = col3 + '_' + c.replace(' ', '')
                cols_list.append(col4)
            elif '          ' not in c:
                col5 = col4 + '_' + c.replace(' ', '')
                cols_list.append(col5)
            elif '            ' not in c:
                col6 = col5 + '_' + c.replace(' ', '')
                cols_list.append(col6)
            elif '              ' not in c:
                col7 = col6 + '_' + c.replace(' ', '')
                cols_list.append(col7)
            elif '                ' not in c:
                col8 = col7 + '_' + c.replace(' ', '')
                cols_list.append(col8)

        df[df.columns[0]] = cols_list
        return df
    
    def getFnCsvStockName(self, df):
        return df[df.columns[0]].iloc[2].replace('종목명 : ', '')

    def getFnCsvFileList(self):
        file_list = os.listdir(self.path)
        tmp = []
        for f in file_list:
            if '.csv' in f:
                tmp.append(f)
        return tmp
    
    def checkFnCsvFileCommon(self):
        tmp = []
        file_list = self.getFnCsvFileList()
        for f in file_list:
            df = self.getFnCsvFile(f)
            df = self.modifyFnCsvFile(df)
            tmp.append(df.columns[2][:7])
        return tmp
    
    def setFnCsvPath(self, path):
        self.path = path
        return
    
    def renameFnCsvFile(self, filename, stock_name):
        if stock_name == '':
            return
        os.rename(self.path+'/'+filename, self.path+'/'+stock_name+'.csv')
        return
    
    def modifyPathFnCsvFile(self, filename):
        df = self.getFnCsvFile(filename)
        gb = df.iloc[5][df.columns[0]].split(' : ')[1]
        date = df.iloc[7][2][:7].replace('.', '_') # 2020.12 
        if gb == '재무상태표':
            shutil.move(self.path +'/'+filename, 'C:/Users/choi97201/Downloads/jaemusangtaepyo/{}'.format(date+filename))
        elif gb == '포괄손익계산서':
            shutil.move(self.path +'/'+filename, 'C:/Users/choi97201/Downloads/pogwalsoniggyesanseo/{}'.format(date+filename))
        elif gb == '재무비율':
            shutil.move(self.path +'/'+filename, 'C:/Users/choi97201/Downloads/jaemubiyul/{}'.format(date+filename))
        else:
            return
        return
    
    
    def getGotCodes(self):
        self.setMaria()
        return list(self.mariaShowData('jasanchonggye_2019_12')['code']) + list(self.mariaShowData('jasanchonggye_2020_12')['code'])
    
    def modifyFnCsvFiles(self):
        self.setFnCsvPath(path='C:/Users/choi97201/Downloads/tmp_csv')
        file_list = self.getFnCsvFileList()
        for f in file_list:
            try:
                self.renameFnCsvFile(filename=f, stock_name=self.getFnCsvStockName(self.getFnCsvFile(filename=f)))
            except:
                pass
        return
    
    def modifyPathFnCsvFiles(self):
        self.setFnCsvPath(path='C:/Users/choi97201/Downloads/tmp_csv')
        file_list = self.getFnCsvFileList()
        for f in file_list:
            try:
                self.modifyPathFnCsvFile(filename=f)
            except:
                pass
        return
    
    def getJaemusangtaepyoWishList(self, wish_list):
        wish_list['자산'] = 'jasanchonggye'
        wish_list['부채'] = 'buchaechonggye'
        wish_list['자산_유동자산'] = 'yudongjasan'
        wish_list['자산_비유동자산'] = 'biyudongjasan'
        wish_list['부채_유동부채'] = 'yudongbuchae'
        wish_list['*발행한주식총수_지배기업주주지분_자본금'] = 'jabongeum'
        wish_list['부채_유동부채_단기차입금'] = 'dangichaibgeum'
        wish_list['부채_비유동부채_장기차입금'] = 'janggichaibgeum'
        wish_list['자산_비유동자산_무형자산'] = 'muhyeongjasan'
        wish_list['자산_유동자산_현금및현금성자산'] = 'hyeongeumseongjasan'
        wish_list['자산_비유동자산_이연법인세자산'] = 'iyeonbeobinsejasan'
        wish_list['부채_비유동부채_이연법인세부채'] = 'iyeonbeobinsebuchae'
        wish_list['*발행한주식총수_지배기업주주지분'] = 'jibaejujujibun'
        wish_list['*발행한주식총수_*보통주'] = 'botongju'
        wish_list['*발행한주식총수_*우선주'] = 'useonju'
        return
    
    def getPogwalsoniggyesanseoWishList(self, wish_list):
        wish_list['매출액(수익)'] = 'maechulaeg' 
        wish_list['영업이익(손실)'] = 'yeongeobiig'
        wish_list['금융원가_이자비용'] = 'ijabiyong'
        wish_list['법인세비용'] = 'beobinsebiyong'
        wish_list['당기순이익(손실)'] = 'danggisuniig'
        wish_list['판매비와관리비_유무형자산상각비_감가상각비'] = 'gamgasanggagbi'
        return
    
    def getJaemubiyulWishList(self, wish_list):
        wish_list['재무비율_수익성_투하자본수익률_ROE(지배주주순이익)'] = 'roe'
        wish_list['상대가치지표_주가배수_수정PER'] = 'per'
        wish_list['상대가치지표_주가배수_수정PBR'] = 'pbr'
        return
    
    def insertFnData(self, gb):
        self.setFnCsvPath('C:/Users/choi97201/Downloads/{}'.format(gb))
        file_list = self.getFnCsvFileList()
        stock_name_df = self.getStockNameCodeDf()
        wish_list = {}
        if gb == 'jaemusangtaepyo':
            self.getJaemusangtaepyoWishList(wish_list)
        elif gb == 'pogwalsoniggyesanseo':
            self.getPogwalsoniggyesanseoWishList(wish_list)
        elif gb == 'jaemubiyul':
            self.getJaemubiyulWishList(wish_list)
        else:
            return
        for f in tqdm(range(len(file_list))):
            stock_name = file_list[f].replace('.csv', '')[7:]
            date = file_list[f][:7]
            try:
                stock_code = stock_name_df[stock_name_df['회사명']==stock_name]['종목코드'].iloc[0]
            except:
                stock_code = '999999'
            df = self.modifyFnCsvFile(df=self.getFnCsvFile(filename=file_list[f]))
            for c in list(wish_list.keys()):
                tablename = 'fnguide_{}_{}'.format(wish_list[c], date)
                dates = [c[:7].replace('.', '_') for c in df.columns[2:]]
                cols = ['code', 'name'] + dates
                dtypes = ['VARCHAR(20)', 'VARCHAR(50)'] + ['FLOAT']*len(dates)
#                 try:
#                     self.mariaSql("DELETE FROM {} WHERE name='999999'".format(tablename))
#                 except:
#                     pass
                empty_check = self.mariaShowData(tablename)
                if empty_check is None:
                    self.mariaCreateTable(tablename, cols, dtypes)
                else:
                    if stock_code == '999999':
                        if stock_name in list(empty_check['code']):
                            continue
                    else:
                        if stock_code in list(empty_check['code']):
                            continue
                
                category_name = df.columns[0]
                try:
                    selected_df = df[df[category_name] == c].reset_index(drop=True)
                    unit = selected_df['단위'].iloc[0]
                    if unit == '천원':
                        mul = 1000
                    elif unit == '주':
                        mul = 1
                    elif unit == '%':
                        mul = 0.01
                    elif unit == '배':
                        mul = 1
                    else:
                        print(unit)

                    data_list = list(selected_df[selected_df.columns[2:]].iloc[0])

                    if stock_code == '999999':
                        data_list = [stock_name, stock_code] + [float(d.replace(',',''))*mul if str(d) != 'nan' and str(d) != 'N/A(IFRS)' and str(d) != '완전잠식' else -123456 for d in data_list]
                    else:
                        data_list = [stock_code, stock_name] + [float(d.replace(',',''))*mul if str(d) != 'nan' and str(d) != 'N/A(IFRS)' and str(d) != '완전잠식' else -123456 for d in data_list]
                    if len(data_list) < 12:
                        data_list = data_list + [-123456]*(12-len(data_list))
                except:
                    continue

                self.mariaInsertData(tablename, tuple(data_list))
                self.mariaCommitDB()
        return
    
    def getStockNameCodeDf(self):
        df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0][['종목코드', '회사명']].reset_index(drop=True)
        
        orgin_list = ['서울식품공업', '삼화콘덴서공업', '삼화전자공업', '미창석유공업', '쌍용양회공업', '한신기계공업',
                   '한국프랜지공업', '태양금속공업', '부산도시가스', '롯데칠성음료', '대림비앤코', '네이블커뮤니케이션즈',
                      '금호석유화학', '계룡건설산업', '티비에이치글로벌', '에스제이엠홀딩스', '에스제이엠',
                      '엔피씨', '엔에이치엔', '엘에스일렉트릭', '케이씨씨', '아이에이치큐', '디아이동일', '쌍용자동차',
                       '휴니드테크놀러지스', '화승인더스트리', '현대엘리베이터', '현대종합상사', '하이트론씨스템즈',
                      '유나이티드', '엘브이엠씨','케이씨티시', '삼영화학공업', '한국석유공업' ,'동양물산기업',
                      '효성 ITX', '현대자동차', '한솔피엔에스', '한국전력공사', '한국수출포장공업', '한국단자공업',
                      '아모레퍼시픽그룹', '신세계I&C', '케이티스카이라이프', '서울도시가스', '삼화페인트공업', 
                      '삼영전자공업', '기아자동차', '교보10호기업인수목적', '에스케이바이오팜', '포스코',
                      '케이티앤지', '케이티', '유진증권', '소마젠', '삼성화재해상보험', '브릿지바이오', '네오이뮨텍'
                      
                     ]
        modified_list = ['서울식품', '삼화콘덴서', '삼화전자', '미창석유', '쌍용양회', '한신기계', 
                         '한국프랜지', '태양금속', '부산가스', '롯데칠성', '대림B&Co', '네이블', 
                         '금호석유', '계룡건설', 'TBH글로벌', 'SJM홀딩스', 'SJM', 
                         'NPC', 'NHN', 'LS ELECTRIC', 'KCC', 'IHQ', 'DI동일', '쌍용차', 
                         '휴니드', '화승인더', '현대엘리베이', '현대상사', '하이트론', 
                         '유나이티드제약', '엘브이엠씨홀딩스', 'KCTC', '삼영화학', '한국석유', '동양물산', 
                         '효성ITX', '현대차', '한솔PNS', '한국전력', '한국수출포장', '한국단자',
                         '아모레G', '신세계 I&C', '스카이라이프', '서울가스', '삼화페인트',
                         '삼영전자', '기아차', '교보10호스팩', 'SK바이오팜', 'POSCO', 
                         'KT&G', 'KT', '유진투자증권', '소마젠(Reg.S)', '삼성화재', '브릿지바이오테라퓨틱스', '네오이뮨텍(Reg.S)'
                        ]
        code_list = []
        for o in orgin_list:
            idx = df[df['회사명'] == o]['종목코드'].index
            code_list.append(df[df['회사명'] == o]['종목코드'].iloc[0])
            df = df.drop(idx)
        df = df.reset_index(drop=True)
        
        orgin_code = list(df['종목코드'])
        orgin_name = list(df['회사명'])
        
        new_code_list = orgin_code + code_list
        new_code_list = [str(c).zfill(6) for c in new_code_list]
        new_name_list = orgin_name + modified_list
        
        df = pd.DataFrame()
        df['종목코드'] = new_code_list
        df['회사명'] = new_name_list
        
        return df
    
    def mariaShowTables(self, sql=None):
        if sql is None:
            self.cur.execute('show tables')
            df = self.cur.fetchall()
            field_names = [i[0] for i in self.cur.description]
            df = pd.DataFrame(df, columns=field_names)
        else:
            try:
                self.cur.execute(sql)
                df = self.cur.fetchall()
                field_names = [i[0] for i in self.cur.description]
                df = pd.DataFrame(df, columns=field_names)
            except Exception as e:
                df = None
        return df
    
    def mergeAllByYear(self, year):
        main.setMaria()
        df = main.mariaShowTables("show tables like 'fnguide%{}_12'".format(year))
        table_list = list(df[df.columns[0]])
        data = {}
        for t in table_list:
            key_name = t.replace('fnguide_', '')
            col_name = key_name.split('_')[0]
            df = main.mariaShowData(t)
            df = df[df.columns[:6]]
            df = df.replace(-123456, '999999')
            df = df.replace('999999', np.nan)
            if col_name in ['botongju', 'jabongeum', 'jasanchonggye', 'yeongeobiig', 'yudongbuchae', 'yudongjasan']:
                df = df.dropna().reset_index(drop=True)
            else:
                df = df.fillna(0).reset_index(drop=True)
            cols = list(df.columns)
            cols = cols[:2] + [col_name+'_'+c for c in cols[2:]]
            df.columns = cols
            data[key_name] = df
            
        df = data[list(data.keys())[0]]
        for k in list(data.keys())[1:]:
            df = pd.merge(df, data[k], on=['code', 'name'])
        return df

In [75]:
df2020

,code,name,beobinsebiyong_2020_12,beobinsebiyong_2019_12,beobinsebiyong_2018_12,beobinsebiyong_2017_12,biyudongjasan_2020_12,biyudongjasan_2019_12,biyudongjasan_2018_12,biyudongjasan_2017_12,...,yeongeobiig_2018_12,yeongeobiig_2017_12,yudongbuchae_2020_12,yudongbuchae_2019_12,yudongbuchae_2018_12,yudongbuchae_2017_12,yudongjasan_2020_12,yudongjasan_2019_12,yudongjasan_2018_12,yudongjasan_2017_12
0,000050,경방,9.657340e+09,1.028710e+10,6.962590e+09,1.026520e+10,1.147570e+12,1.105880e+12,1.181080e+12,1.183790e+12,...,4.097980e+10,4.230920e+10,2.856410e+11,1.923800e+11,1.631520e+11,3.194770e+11,1.173310e+11,2.648610e+11,1.207110e+11,9.025470e+10
1,000250,삼천당제약,9.085850e+09,-2.824420e+08,-1.315290e+08,3.593120e+09,1.505510e+11,1.276580e+11,1.188870e+11,9.893550e+10,...,1.550010e+10,2.396880e+10,2.365830e+10,4.520600e+10,3.841910e+10,2.369930e+10,1.665280e+11,1.624930e+11,1.422720e+11,1.312080e+11
2,000270,기아차,3.537730e+11,7.044450e+11,3.127010e+11,1.720350e+11,3.439710e+13,3.378940e+13,3.207480e+13,3.065240e+13,...,1.157470e+12,6.622260e+11,2.109760e+13,1.727660e+13,1.483470e+13,1.532300e+13,2.609340e+13,2.155540e+13,1.971180e+13,2.164210e+13
3,000390,삼화페인트,2.412110e+09,2.447930e+09,2.073900e+08,2.534120e+09,2.849290e+11,2.792730e+11,2.796470e+11,2.783310e+11,...,7.883940e+09,8.769700e+09,2.079050e+11,2.176570e+11,1.529390e+11,1.544720e+11,3.052140e+11,2.758290e+11,2.751850e+11,2.621720e+11
4,000520,삼일제약,-1.425040e+08,6.693470e+08,4.276120e+08,5.549640e+08,1.191380e+11,1.046750e+11,8.876430e+10,8.331690e+10,...,-5.731790e+09,1.307020e+09,9.868370e+10,6.815570e+10,8.531990e+10,5.241200e+10,7.252340e+10,7.919980e+10,5.918430e+10,5.216020e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,326030,SK바이오팜,7.504280e+09,-1.293160e+10,5.831990e+08,6.272980e+08,6.563880e+10,4.039240e+10,1.397740e+10,1.653920e+10,...,-1.391030e+11,-9.893260e+10,7.059550e+10,1.456410e+11,3.097030e+10,4.697840e+10,4.346030e+11,9.868070e+10,8.155810e+10,8.182810e+10
287,339950,아이비김영,1.091770e+09,1.633510e+09,-1.261600e+09,4.719300e+07,4.348750e+10,3.148340e+10,1.756280e+10,1.163070e+10,...,5.902430e+09,1.718300e+09,1.886310e+10,1.598010e+10,1.197770e+10,1.168600e+10,1.598530e+10,1.045090e+10,7.539090e+09,5.489200e+09
288,347890,엠투아이,1.996470e+09,1.040370e+09,7.194010e+08,9.662020e+08,2.974770e+10,1.773830e+10,1.435160e+10,1.414270e+10,...,5.152640e+09,6.073070e+09,4.637080e+09,3.651590e+09,2.300770e+09,4.362180e+09,3.658570e+10,2.528890e+10,2.581470e+10,2.292640e+10
289,348210,넥스틴,-5.300810e+08,1.175970e+08,3.405100e+07,2.587900e+07,8.923290e+09,3.766540e+09,4.794730e+09,6.163470e+09,...,-4.154810e+08,1.731620e+09,7.637760e+09,1.532310e+10,1.156890e+10,2.136700e+10,4.533600e+10,1.469950e+10,1.423970e+10,1.244900e+10
